In [1]:
import numpy as np
import pandas as pd

In [2]:
def calcular_inventario(A,B,k):
    inva = np.linalg.inv(A.to_numpy()) 
    bma = np.matmul(B.to_numpy(), inva)
    M = np.matmul(bma, k.to_numpy())
    
    return inva, bma, M

## Elabore a matriz  Atil e Btil utilizando os seguintes dados:

### o primeiro processo produz 1 kg de coque, consome 0,04 kg de aço, utiliza 1 kg de carvão (da natureza) e emite 0,5 kg de CO2 e 2 g de NOx para o ar,
### o segundo processo produz 3 kg de aço e 1 kg de ferro fundido e, para tanto, consome 0,4 kg de coque, e emite 2 kg de CO2 e 7 kg de NOx para o meio ambiente.

In [3]:
# Matriz A: Fluxos intermediários (tecnológicos)
coque = {'Produto':'Coque', 'Processo 1':1, 'Processo 2':-0.4}
aco = {'Produto':'Aço', 'Processo 1': -0.04, 'Processo 2': 3}
fofo = {'Produto':'Fofo', 'Processo 1': 0, 'Processo 2': 1}
linhas = [coque, aco, fofo]
nprods = len(linhas)

dfA = pd.DataFrame()
for linha in linhas:
    dfA = dfA.append(linha, ignore_index=True)
dfA = dfA.set_index('Produto')
dfA

,Processo 1,Processo 2
Produto,,
Coque,1.00,-0.4
Aço,-0.04,3.0
Fofo,0.00,1.0


In [4]:
# Matriz B: Fluxos elementares
carvao = {'Produto':'Carvão', 'Processo 1': -1, 'Processo 2': 0}
co2 = {'Produto':'CO2', 'Processo 1': 0.5, 'Processo 2': 2}
nox = {'Produto':'NOx', 'Processo 1': 0.002, 'Processo 2': 7}
linhas = [carvao, co2, nox]

dfB = pd.DataFrame()
for linha in linhas:
    dfB = dfB.append(linha, ignore_index=True)
dfB = dfB.set_index('Produto')
dfB

,Processo 1,Processo 2
Produto,,
Carvão,-1.000,0.0
CO2,0.500,2.0
NOx,0.002,7.0


In [5]:
# k é o fluxo de referência
kcoque = {'Produto':'Coque', 'Valor':0}
kaco = {'Produto':'Aço', 'Valor': 1610}
kfofo = {'Produto':'Fofo', 'Valor':0 }
linhas = [kcoque, kaco, kfofo]

dfk = pd.DataFrame()
for linha in linhas:
    dfk = dfk.append(linha, ignore_index=True)
dfk = dfk.set_index('Produto')
dfk

,Valor
Produto,
Coque,0.0
Aço,1610.0
Fofo,0.0


# Tem um problema de a matriz A não ser inversa. É um problema de alocação

### OPÇÃO 1: COMPLETAR COM LINHA DA MATRIZ IDENTIDADE

### OPÇÃO 2: JOGAR O AÇO PARA FLUXO ELEMENTAR

### OPÇÃO 3: SUBDIVIDIR UM PROCESSO E ALOCAR COM POR MASSA

# OPÇÃO 1

In [6]:
dfa1 = dfA.copy()
dfa1['Processo 3'] = 0
dfa1.at['Fofo', 'Processo 3'] = 1
dfa1

,Processo 1,Processo 2,Processo 3
Produto,,,
Coque,1.00,-0.4,0
Aço,-0.04,3.0,0
Fofo,0.00,1.0,1


In [7]:
dfb1 = dfB.copy()
dfb1['Processo 3']=0
dfb1

,Processo 1,Processo 2,Processo 3
Produto,,,
Carvão,-1.000,0.0,0
CO2,0.500,2.0,0
NOx,0.002,7.0,0


In [8]:
dfk1 = dfk.copy()

In [9]:
inva, bma, M = calcular_inventario(dfa1,dfb1,dfk1)

Emissão de Carvão: -215.82
Emissão de CO2: 1187.0
Emissão de NOx: 3777.24


# OPÇÃO 2

In [10]:
# retirar o aço do A e colocar como um B
retirado = 'Fofo'

In [11]:
dfa2 = dfA[dfA.index!=retirado]
dfa2

,Processo 1,Processo 2
Produto,,
Coque,1.00,-0.4
Aço,-0.04,3.0


In [12]:
dfk2 = dfk[dfk.index!=retirado]
dfk2

,Valor
Produto,
Coque,0.0
Aço,1610.0


In [13]:
dfb2 = dfB.copy()
dfb2 = dfb2.append(dfA.loc[[retirado]])
dfb2

,Processo 1,Processo 2
Produto,,
Carvão,-1.000,0.0
CO2,0.500,2.0
NOx,0.002,7.0
Fofo,0.000,1.0


In [14]:
inva, bma, M = calcular_inventario(dfa2,dfb2,dfk2)

Emissão de Carvão: -215.82
Emissão de CO2: 1187.0
Emissão de NOx: 3777.24
Emissão de Fofo: 539.54


# OPÇÃO 3

In [15]:
dfb3 = dfB.copy()
dfk3 = dfk.copy()

In [16]:
# Dividir um processo em 2 subprocessos e alocar por massa

In [17]:
# Vamos dividir o processo 2 em aço e ferro fundido
# O novo processo 2 vai ter tudo de aço, 0 de fofo, e a proporção entre aço e fofo do antigo processo 2 para os outros produtos
# O novo processo 3 vai ter 0 de aço, tudo de fofo, e a proporção entre aço e fofo do antigo processo 2 para os outros produtos

In [18]:
qtd_aco = dfA.loc['Aço', 'Processo 2']

qtd_fofo = dfA.loc['Fofo', 'Processo 2']
tot = qtd_aco + qtd_fofo

prop_aco = qtd_aco / tot
prop_fofo = qtd_fofo / tot

In [19]:
dfa3 = dfA.copy()

dfa3['Processo 3'] = dfa3['Processo 2']*prop_fofo # vai ser do fofo
dfa3.at['Aço', 'Processo 3'] = 0
dfa3.at['Fofo', 'Processo 3'] = qtd_fofo

dfa3['Processo 2'] = dfa3['Processo 2']*prop_aco # vai ser do aço
dfa3.at['Aço', 'Processo 2'] = qtd_aco
dfa3.at['Fofo', 'Processo 2'] = 0


dfa3

,Processo 1,Processo 2,Processo 3
Produto,,,
Coque,1.00,-0.3,-0.1
Aço,-0.04,3.0,0.0
Fofo,0.00,0.0,1.0


In [20]:
dfb3 = dfB.copy()

dfb3['Processo 3'] = dfb3['Processo 2']*prop_fofo # vai ser do aço
dfb3['Processo 2'] = dfb3['Processo 2']*prop_aco # vai ser do aço

dfb3

,Processo 1,Processo 2,Processo 3
Produto,,,
Carvão,-1.000,0.00,0.00
CO2,0.500,1.50,0.50
NOx,0.002,5.25,1.75


In [21]:
dfk3 = dfk.copy()

In [22]:
inva, bma, M = calcular_inventario(dfa3,dfb3,dfk3)

Emissão de Carvão: -161.65
Emissão de CO2: 889.06
Emissão de NOx: 2829.14


In [ ]:
# mudanças climaticas, acidificacao, toxicidade humana
impactos = {'CO2':[1, 0, 0], }
a)	o CO2 contribui com fator 1 para mudanças climáticas (kg eq. CO2) e o CH4 com fator 25;
b)	o SO2 apresenta fator 1 para acidificação (kg eq. SO2) e o NO2, 0,7 e,
c)	para a toxicidade humana, o MP tem fator 0,2 e o SO2, fator 0,01